In [10]:
import sys
import os

# Thêm thư mục src vào sys.path (đường dẫn tuyệt đối từ notebook)
sys.path.append(os.path.abspath('../src'))

from core.db.qdrant import QdrantDatabaseConnector
from core.config import settings
from qdrant_client.http.exceptions import UnexpectedResponse
import traceback

In [11]:
# Nếu chạy từ notebook (ngoài container)
settings.patch_localhost()
# Kết nối Qdrant
qdrant = QdrantDatabaseConnector()

HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"


In [12]:
# Đọc dữ liệu từ cleaned_exams
result = qdrant.scroll("cleaned_exams", limit=10)
points = result[0]  # List các points

HTTP Request: POST http://localhost:6333/collections/cleaned_exams/points/scroll "HTTP/1.1 200 OK"


In [13]:
points

[Record(id='9f43a64b-676c-4268-b8a8-fccd7c2b358b', payload={'type': 'exam', 'link': 'https://loigiaihay.com/de-thi-vao-lop-6-mon-toan-truong-nguyen-tat-thanh-nam-2025-co-dap-an-a185630.html', 'cleaned_content': '**Đáp án**\n**HƯỚNG DẪN GIẢI CHI TIẾT**\n**I. TRẮC NGHIỆM**\n**Câu 1****(0,5 điểm)****.** Cho biểu đồ như hình dưới đây. Tìm tỉ số phần trăm số học sinh khối 9 so với số học sinh toàn trường.\n![](https://img.loigiaihay.com/picture/2025/0602/20_1.png)\n**A.** 22,5% **B.** 12,5% **C.** 32% **D.** 25%\n**Cách giải:**\nSố học sinh khối 6 được biểu diễn là 14\nTỉ số phần trăm học sinh khối 9 là: 100% - (12,5% + 40% + 25%) = 22,5%\n**Đáp số: 22,5%**\n**Câu 2****(0,5 điểm)****.** Gieo 1 viên xúc sắc 20 lần liên tiếp kết quả nhận được như bảng sau:\n![](https://img.loigiaihay.com/picture/2025/0602/20_1.PNG)\nTỉ số giữa số lần xuất hiện mặt có số chấm là số lẻ và tổng số lần đã gieo là bao nhiêu?\n**A.** 920**B.** 1120**C.** 320**D.** 120\n**Cách giải:**\nSố lần xuất hiện mặt có chấm l

In [14]:
cleaned_content = points[0].payload["cleaned_content"]
print(cleaned_content)

**Đáp án**
**HƯỚNG DẪN GIẢI CHI TIẾT**
**I. TRẮC NGHIỆM**
**Câu 1****(0,5 điểm)****.** Cho biểu đồ như hình dưới đây. Tìm tỉ số phần trăm số học sinh khối 9 so với số học sinh toàn trường.
![](https://img.loigiaihay.com/picture/2025/0602/20_1.png)
**A.** 22,5% **B.** 12,5% **C.** 32% **D.** 25%
**Cách giải:**
Số học sinh khối 6 được biểu diễn là 14
Tỉ số phần trăm học sinh khối 9 là: 100% - (12,5% + 40% + 25%) = 22,5%
**Đáp số: 22,5%**
**Câu 2****(0,5 điểm)****.** Gieo 1 viên xúc sắc 20 lần liên tiếp kết quả nhận được như bảng sau:
![](https://img.loigiaihay.com/picture/2025/0602/20_1.PNG)
Tỉ số giữa số lần xuất hiện mặt có số chấm là số lẻ và tổng số lần đã gieo là bao nhiêu?
**A.** 920**B.** 1120**C.** 320**D.** 120
**Cách giải:**
Số lần xuất hiện mặt có chấm là số lẻ là: 5 + 3 + 1 = 9 (lần)
Tỉ số giữa số lần xuất hiện mặt số chấm là số lẻ và tổng số lần đã gieo là: 9:20=920
**Đáp số:** 920
**Câu 3 (0,5 điểm).** Tìm số tự nhiên thích hợp điền vào chỗ trống là: 526<....26<625
**A.** 3

In [22]:
import os
from typing import List, Optional, Dict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

class Problem(BaseModel):
    question: str = Field(..., description="Câu hỏi chính của bài thi, thường bắt đầu bằng 'Câu ...'. Câu hỏi có thể chứa các công thức latex")
    image_url: Optional[str] = Field(None, description="URL hình ảnh minh họa, trong trường hợp câu hỏi cần hình vẽ để mô tả")
    solution: str = Field(..., description="Hướng dẫn giải, trong hướng dẫn có thể có các công thức latex")
    result: Optional[str] = Field(None, description="Đáp án của những câu hỏi trắc nghiệm")
    
class Exam(BaseModel):
    """Mô hình dữ liệu cho toàn bộ đề thi, chứa một danh sách các bài toán."""
    problems: List[Problem]

In [42]:
llm = ChatOpenAI(
  api_key=os.getenv("OPEN_ROUTER"),
  base_url="https://openrouter.ai/api/v1",
  model="qwen/qwen3-8b:free",
)

# Khởi tạo Output Parser với Pydantic model đã định nghĩa
# Parser này sẽ tự động tạo hướng dẫn định dạng và phân tích cú pháp đầu ra của LLM
parser = PydanticOutputParser(pydantic_object=Exam)

prompt_template = """
Bạn là một trợ lý AI chuyên phân tích và bóc tách nội dung đề thi.
Nhiệm vụ của bạn là đọc kỹ văn bản đề thi dưới đây và trích xuất tất cả các bài tập vào một cấu trúc JSON.
Mỗi bài tập phải có đầy đủ các trường thông tin được yêu cầu. Chú ý các bài tự luận có thể không có lựa chọn (choices).
NOTE: GIỮ CÁC CÔNG THỨC Ở DẠNG LATEX

{format_instructions}

Văn bản đề thi cần xử lý:
---------------------
{exam_text}
---------------------
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["exam_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Kết hợp các thành phần lại thành một chuỗi (chain)
# Cú pháp `|` là LangChain Expression Language (LCEL)
chain = prompt | llm | parser


In [43]:
result = chain.invoke({"exam_text": cleaned_content})

HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}

In [38]:
result

Exam(problems=[Problem(question='Câu 1 (0,5 điểm). Cho biểu đồ như hình dưới đây. Tìm tỉ số phần trăm số học sinh khối 9 so với số học sinh toàn trường.', image_url='https://img.loigiaihay.com/picture/2025/0602/20_1.png', solution='Số học sinh khối 6 được biểu diễn là 14\nTỉ số phần trăm học sinh khối 9 là: 100% - (12,5% + 40% + 25%) = 22,5%', result='22,5%'), Problem(question='Câu 2 (0,5 điểm). Gieo 1 viên xúc sắc 20 lần liên tiếp kết quả nhận được như bảng sau: \n![](https://img.loigiaihay.com/picture/2025/0602/20_1.PNG)\nTỉ số giữa số lần xuất hiện mặt có số chấm là số lẻ và tổng số lần đã gieo là bao nhiêu?', image_url='https://img.loigiaihay.com/picture/2025/0602/20_1.PNG', solution='Số lần xuất hiện mặt có chấm là số lẻ là: 5 + 3 + 1 = 9 (lần)\nTỉ số giữa số lần xuất hiện mặt số chấm là số lẻ và tổng số lần đã gieo là: 9:20=920', result='920'), Problem(question='Câu 3 (0,5 điểm). Tìm số tự nhiên thích hợp điền vào chỗ trống là: 526<....26<625', image_url=None, solution='Số tự nhi

In [39]:
print(result.problems[0].question)

Câu 1 (0,5 điểm). Cho biểu đồ như hình dưới đây. Tìm tỉ số phần trăm số học sinh khối 9 so với số học sinh toàn trường.


In [40]:
print(result.problems[0].solution)

Số học sinh khối 6 được biểu diễn là 14
Tỉ số phần trăm học sinh khối 9 là: 100% - (12,5% + 40% + 25%) = 22,5%


In [41]:
print(result.problems[0].result)

22,5%
